In [1]:
from bs4 import BeautifulSoup as bs
import urllib
import requests
import lxml
import pandas as pd
import datetime
from dateutil.parser import parse
import re

In [2]:
def get_name_place(soup):    
    li = soup.find_all('strong', class_ = 'rvw-aut__inf-nm')    # Scrapping tags containing name & place
    chk = []
    for i in li:
        chk.append(i.text)                                      # Getting the texts out the obtained tags 

    name = []                                                   # defined empty list for reviewer names 
    place = []                                                  # defined empty list for reviewer locations
    for i in chk:
        if ' of ' in i:
            name.append(i.split(' of ')[0])
            place.append(i.split(' of ')[1])
    return name,place




def get_reviews(soup):
    rv = soup.find_all('div',class_='rvw-bd')
    revw = []
    for i in range(0,len(rv)):
        revw.append(rv[i].get_text())

    for i in range(0,len(revw)):
        revw[i] = revw[i].rstrip("\n                    Read full review\n")

    for i in range(0,len(revw)):
        revw[i] = re.split(', 2020|, 2019|, 2018|, 2017|, 2016|, 2015|, 2014|, 2013|, 2012|, 2011|, 2010',revw[i])[1]
    return revw



def get_ratings(soup):
    imgs = [div.find('img') for div in soup.find_all('div',{'class':'rvw__hdr-stat'}) if div.find('img')]
    rating_raw = [img.get('alt') for img in imgs]
    rating = [int(rating_raw[rate].split(' ')[2]) for rate in range(0,len(rating_raw))]
    return rating


def get_date(soup):
    date_elem = soup.find_all('span', class_='ca-txt-cpt')

    date_raw = []
    for i in range(0,len(date_elem)):
        date_raw.append(date_elem[i].get_text())

    dates_ = []
    for i in range(0,len(date_raw)):
        dates_.append(date_raw[i].split(': ')[1])


    date_in_str_format = []
    for i in dates_:
        date_in_str_format.append(i.replace('Sept.','Sep.'))


    date_in_datetime_format = []
    for j in date_in_str_format:
        date_in_datetime_format.append(parse(j).date())

    return date_in_datetime_format    

In [3]:
def main(link1):
    page1 = requests.get(link1)
    soup1 = bs(page1.content,'html')
    
    
    name_col, place_col = get_name_place(soup1)
    revs_col = get_reviews(soup1)
    ratings_col = get_ratings(soup1)
    dates_col = get_date(soup1)
    
    df = pd.DataFrame({'Date':dates_col,'author':name_col,'location':place_col,'reviews':revs_col,'Rating':ratings_col})
    return df

In [4]:
DF_list = list()
for page_num in range(2,100):
    try:
        link = 'https://www.consumeraffairs.com/travel/booking_com.html?page='+str(page_num)
        DF_list.append(main(link))
        print(page_num)

    except (ValueError):
        continue  

result = pd.concat(DF_list)
print(result.shape)
result

2
3
4
7
8
10
12
13
14
15
17
18
19
20
21
22
23
24
26
28
29
30
31
32
33
35
37
38
39
40
41
42
43
44
45
46
47
48
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
67
68
69
70
71
73
74
75
76
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
97
98
(2390, 5)


,Date,author,location,reviews,Rating
0,2020-10-16,Jason,"Elizabethton, TN",I rented a car and paid in advance 4 months ag...,1
1,2020-10-16,Jann,"Australia, Other","Ordinarily, BOOKING.COM do a reasonable job, B...",2
2,2020-10-15,Mark,"Renfrew, Other",Worse Customer services department I have ever...,1
3,2020-10-15,Veronica,"Wellesley Hills, MA",I booked a room for my family of 5. The hotel ...,1
4,2020-10-11,Jan,"Rehoboth, MA",After confirming a reservation for the Thanksg...,1
...,...,...,...,...,...
25,2015-11-23,Chris,"St. John'S, NL","Using the Booking.com website, which asked for...",1
26,2015-11-20,Ann,"Tulsa, Other",I made a reservation on Booking.com on 11-15-1...,1
27,2015-11-19,Xuemei (Ruby),"Bellevue, WA",I booked and got e-mail confirmation. Shows on...,1
28,2015-11-18,James,"Haines City, FL","Booked a good rate, $69 about 3 weeks in advan...",1


In [12]:
# testpage = 'https://www.consumeraffairs.com/travel/booking_com.html?page=5'
# testdata  = main(testpage)
# testdata

In [11]:
result.to_csv("Customer_reviews.csv") 

In [13]:
data = result

In [14]:
data.head()

,Date,author,location,reviews,Rating
0,2020-10-16,Jason,"Elizabethton, TN",I rented a car and paid in advance 4 months ag...,1
1,2020-10-16,Jann,"Australia, Other","Ordinarily, BOOKING.COM do a reasonable job, B...",2
2,2020-10-15,Mark,"Renfrew, Other",Worse Customer services department I have ever...,1
3,2020-10-15,Veronica,"Wellesley Hills, MA",I booked a room for my family of 5. The hotel ...,1
4,2020-10-11,Jan,"Rehoboth, MA",After confirming a reservation for the Thanksg...,1


In [16]:
data.location.nunique()

1636

In [21]:
data2 = pd.DataFrame(data.location.str.split(', ',1).tolist(),
                                 columns = ['location','state'])

In [22]:
data2

,location,state
0,Elizabethton,TN
1,Australia,Other
2,Renfrew,Other
3,Wellesley Hills,MA
4,Rehoboth,MA
...,...,...
2385,St. John'S,NL
2386,Tulsa,Other
2387,Bellevue,WA
2388,Haines City,FL


In [24]:
data[['location', 'state']] = data['location'].str.split(' ', 1, expand=True)

In [25]:
data

,Date,author,location,reviews,Rating,state
0,2020-10-16,Jason,"Elizabethton,",I rented a car and paid in advance 4 months ag...,1,TN
1,2020-10-16,Jann,"Australia,","Ordinarily, BOOKING.COM do a reasonable job, B...",2,Other
2,2020-10-15,Mark,"Renfrew,",Worse Customer services department I have ever...,1,Other
3,2020-10-15,Veronica,Wellesley,I booked a room for my family of 5. The hotel ...,1,"Hills, MA"
4,2020-10-11,Jan,"Rehoboth,",After confirming a reservation for the Thanksg...,1,MA
...,...,...,...,...,...,...
25,2015-11-23,Chris,St.,"Using the Booking.com website, which asked for...",1,"John'S, NL"
26,2015-11-20,Ann,"Tulsa,",I made a reservation on Booking.com on 11-15-1...,1,Other
27,2015-11-19,Xuemei (Ruby),"Bellevue,",I booked and got e-mail confirmation. Shows on...,1,WA
28,2015-11-18,James,Haines,"Booked a good rate, $69 about 3 weeks in advan...",1,"City, FL"
